In [1]:
# libraries
from dotenv import load_dotenv

# load environment variables from .env file
_ = load_dotenv("../.env")

from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver

model = ChatOpenAI(
    api_key="ollama",
    model="llama3.2",
    base_url="http://localhost:11434/v1",
)

from uuid import uuid4
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage

"""
In previous examples we've annotated the `messages` state key
with the default `operator.add` or `+` reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, we annotate the
`messages` key with a customer reducer function, which replaces
messages with the same `id`, and appends them otherwise.
"""


def reduce_messages(
    left: list[AnyMessage], right: list[AnyMessage]
) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged


class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [2]:
tool = TavilySearchResults(max_results=2)

In [3]:
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm", self.exists_action, {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer, interrupt_before=["action"]
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state["messages"]
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {"messages": [message]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state["messages"][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state["messages"][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t["name"]].invoke(t["args"])
            results.append(
                ToolMessage(tool_call_id=t["id"], name=t["name"], content=str(result))
            )
        print("Back to the model!")
        return {"messages": results}

### Manual human approval

In [4]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
with SqliteSaver.from_conn_string(":memory:") as checkpointer:
    abot = Agent(model, [tool], system=prompt, checkpointer=checkpointer)

In [5]:
messages = [HumanMessage(content="Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in SF?', id='2a1574d0-3132-4bd6-8c94-da074c467826'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_umlw1l1s', 'function': {'arguments': '{"query":"San Francisco weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 257, 'total_tokens': 280, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-64e9bccf-4789-47c8-97a7-a9ca79ad8ffe-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather'}, 'id': 'call_umlw1l1s', 'type': 'tool_call'}], usage_metadata={'input_tokens': 257, 'output_tokens': 23, 'total_tokens': 280})]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_umlw1l1s', 'function': 

In [7]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', id='2a1574d0-3132-4bd6-8c94-da074c467826'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_umlw1l1s', 'function': {'arguments': '{"query":"San Francisco weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 257, 'total_tokens': 280, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-64e9bccf-4789-47c8-97a7-a9ca79ad8ffe-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather'}, 'id': 'call_umlw1l1s', 'type': 'tool_call'}], usage_metadata={'input_tokens': 257, 'output_tokens': 23, 'total_tokens': 280}), HumanMessage(content='Whats the weather in SF?', id='cd9ad17f-3662-4b5f-b526-4a2f6fcea

In [9]:
abot.graph.get_state(thread).next

('action',)

In [10]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco current weather'}, 'id': 'call_c1x2hsk2', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.775, \'lon\': -122.4183, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1739777204, \'localtime\': \'2025-02-16 23:26\'}, \'current\': {\'last_updated_epoch\': 1739776500, \'last_updated\': \'2025-02-16 23:15\', \'temp_c\': 11.1, \'temp_f\': 52.0, \'is_day\': 0, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/116.png\', \'code\': 1003}, \'wind_mph\': 4.5, \'wind_kph\': 7.2, \'wind_degree\': 271, \'wind_dir\': \'W\', \'pressure_mb\': 1020.0, \'pressure_in\': 30.13, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 97, \'cloud\': 75, \'feelslike_c\

In [12]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', id='2a1574d0-3132-4bd6-8c94-da074c467826'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_umlw1l1s', 'function': {'arguments': '{"query":"San Francisco weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 257, 'total_tokens': 280, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-64e9bccf-4789-47c8-97a7-a9ca79ad8ffe-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather'}, 'id': 'call_umlw1l1s', 'type': 'tool_call'}], usage_metadata={'input_tokens': 257, 'output_tokens': 23, 'total_tokens': 280}), HumanMessage(content='Whats the weather in SF?', id='cd9ad17f-3662-4b5f-b526-4a2f6fcea

In [11]:
abot.graph.get_state(thread).next

()

### Modify State

In [5]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='88d8485f-fabe-4186-8b7d-066710061c75'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nldupa0c', 'function': {'arguments': '{"query":"LA weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 257, 'total_tokens': 279, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2e87ed19-0ba0-4c3c-a57a-38fa90fc5eca-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'LA weather'}, 'id': 'call_nldupa0c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 257, 'output_tokens': 22, 'total_tokens': 279})]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nldupa0c', 'function': {'arguments': '{"query

In [24]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='a9627b24-af6f-4727-9337-8009b7856818'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wmj5qxzg', 'function': {'arguments': '{"query":"LA weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 257, 'total_tokens': 279, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a1049621-8f26-43d1-ab42-148669ad90f0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'LA weather'}, 'id': 'call_wmj5qxzg', 'type': 'tool_call'}], usage_metadata={'input_tokens': 257, 'output_tokens': 22, 'total_tokens': 279})]}, next=('action',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efed0

In [16]:
current_values = abot.graph.get_state(thread)
current_values.values["messages"][-1]

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_re03dxhy', 'function': {'arguments': '{"query":"LA weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 257, 'total_tokens': 279, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c4f224e9-d987-48d1-b46f-4b92ef93fae9-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'LA weather'}, 'id': 'call_re03dxhy', 'type': 'tool_call'}], usage_metadata={'input_tokens': 257, 'output_tokens': 22, 'total_tokens': 279})

In [17]:
current_values.values["messages"][-1].tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'LA weather'},
  'id': 'call_re03dxhy',
  'type': 'tool_call'}]

In [25]:
_id = current_values.values["messages"][-1].tool_calls[0]["id"]
current_values.values["messages"][-1].tool_calls = [
    {
        "name": "tavily_search_results_json",
        "args": {"query": "current weather in Louisiana"},
        "id": _id,
        "type": "tool_call",
    }
]

In [26]:
current_values.values["messages"][-1].tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in Louisiana'},
  'id': 'call_re03dxhy',
  'type': 'tool_call'}]

In [27]:
abot.graph.update_state(thread, current_values.values)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='a9627b24-af6f-4727-9337-8009b7856818'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wmj5qxzg', 'function': {'arguments': '{"query":"LA weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 257, 'total_tokens': 279, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a1049621-8f26-43d1-ab42-148669ad90f0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'LA weather'}, 'id': 'call_wmj5qxzg', 'type': 'tool_call'}], usage_metadata={'input_tokens': 257, 'output_tokens': 22, 'total_tokens': 279}), HumanMessage(content='Whats the weather in LA?', id='410da7e8-2ce7-46da-8be5-05b33f394fb8'), AIMessage(content='', additional_kwa

{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1efed03a-1d72-6efb-8002-2fe1d526a098'}}

In [28]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='a9627b24-af6f-4727-9337-8009b7856818'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wmj5qxzg', 'function': {'arguments': '{"query":"LA weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 257, 'total_tokens': 279, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a1049621-8f26-43d1-ab42-148669ad90f0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'LA weather'}, 'id': 'call_wmj5qxzg', 'type': 'tool_call'}], usage_metadata={'input_tokens': 257, 'output_tokens': 22, 'total_tokens': 279}), HumanMessage(content='Whats the weather in LA?', id='410da7e8-2ce7-46da-8be5-05b33f394fb8'), AIMessage(conte

In [29]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_re03dxhy', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Louisiana\', \'region\': \'Missouri\', \'country\': \'USA United States of America\', \'lat\': 39.4411, \'lon\': -91.0551, \'tz_id\': \'America/Chicago\', \'localtime_epoch\': 1739778620, \'localtime\': \'2025-02-17 01:50\'}, \'current\': {\'last_updated_epoch\': 1739778300, \'last_updated\': \'2025-02-17 01:45\', \'temp_c\': -11.6, \'temp_f\': 11.1, \'is_day\': 0, \'condition\': {\'text\': \'Clear\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/113.png\', \'code\': 1000}, \'wind_mph\': 10.1, \'wind_kph\': 16.2, \'wind_degree\': 319, \'wind_dir\': \'NW\', \'pressure_mb\': 1028.0, \'pressure_in\': 30.36, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 67, \'cloud\': 0, \'feelslike_c\': -19.0, \'

### Time Travel

In [30]:
states = []
for state in abot.graph.get_state_history(thread):
    print(state)
    print("--")
    states.append(state)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='a9627b24-af6f-4727-9337-8009b7856818'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wmj5qxzg', 'function': {'arguments': '{"query":"LA weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 257, 'total_tokens': 279, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a1049621-8f26-43d1-ab42-148669ad90f0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'LA weather'}, 'id': 'call_wmj5qxzg', 'type': 'tool_call'}], usage_metadata={'input_tokens': 257, 'output_tokens': 22, 'total_tokens': 279}), HumanMessage(content='Whats the weather in LA?', id='410da7e8-2ce7-46da-8be5-05b33f394fb8'), AIMessage(conte

In [31]:
to_replay = states[-3]
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='a9627b24-af6f-4727-9337-8009b7856818'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wmj5qxzg', 'function': {'arguments': '{"query":"LA weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 257, 'total_tokens': 279, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a1049621-8f26-43d1-ab42-148669ad90f0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'LA weather'}, 'id': 'call_wmj5qxzg', 'type': 'tool_call'}], usage_metadata={'input_tokens': 257, 'output_tokens': 22, 'total_tokens': 279})]}, next=('action',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efed0

In [ ]:
for event in abot.graph.stream(None, to_replay.config):
    for k, v in event.items():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'LA weather'}, 'id': 'call_wmj5qxzg', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Los Angeles\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 34.0522, \'lon\': -118.2428, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1739779140, \'localtime\': \'2025-02-16 23:59\'}, \'current\': {\'last_updated_epoch\': 1739778300, \'last_updated\': \'2025-02-16 23:45\', \'temp_c\': 12.8, \'temp_f\': 55.0, \'is_day\': 0, \'condition\': {\'text\': \'Clear\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/113.png\', \'code\': 1000}, \'wind_mph\': 2.2, \'wind_kph\': 3.6, \'wind_degree\': 169, \'wind_dir\': \'S\', \'pressure_mb\': 1015.0, \'pressure_in\': 29.97, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 83, \'cloud\': 0, \'feelslike_c\': 13.4, \'feelslike_f\': 56.

### Go back in time and edit

In [33]:
to_replay.values["messages"][-1].tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'LA weather'},
  'id': 'call_wmj5qxzg',
  'type': 'tool_call'}]

In [34]:
_id = to_replay.values["messages"][-1].tool_calls[0]["id"]
to_replay.values["messages"][-1].tool_calls = [
    {
        "name": "tavily_search_results_json",
        "args": {"query": "current weather in LA, accuweather"},
        "id": _id,
        'type': 'tool_call'
    }
]

In [36]:
branch_state = abot.graph.update_state(to_replay.config, to_replay.values)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='a9627b24-af6f-4727-9337-8009b7856818'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wmj5qxzg', 'function': {'arguments': '{"query":"LA weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 257, 'total_tokens': 279, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a1049621-8f26-43d1-ab42-148669ad90f0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'call_wmj5qxzg', 'type': 'tool_call'}], usage_metadata={'input_tokens': 257, 'output_tokens': 22, 'total_tokens': 279})]}


In [37]:
for event in abot.graph.stream(None, branch_state):
    for k, v in event.items():
        if k != "__end__":
            print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'call_wmj5qxzg', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Los Angeles\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 34.0522, \'lon\': -118.2428, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1739779406, \'localtime\': \'2025-02-17 00:03\'}, \'current\': {\'last_updated_epoch\': 1739779200, \'last_updated\': \'2025-02-17 00:00\', \'temp_c\': 12.8, \'temp_f\': 55.0, \'is_day\': 0, \'condition\': {\'text\': \'Clear\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/113.png\', \'code\': 1000}, \'wind_mph\': 2.2, \'wind_kph\': 3.6, \'wind_degree\': 143, \'wind_dir\': \'SE\', \'pressure_mb\': 1015.0, \'pressure_in\': 29.97, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 83, \'cloud\': 0, \'feelslike_c\': 1

### Add message to a state at a given time

In [19]:
messages = [HumanMessage(content="Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "3"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in SF?', id='612d7ee1-8c31-4fd1-bb21-74e86f2f5b8c'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_psv901bm', 'function': {'arguments': '{"query":"SF weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 257, 'total_tokens': 275, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2ba8bf57-6764-4df5-931b-bcf67ea38043-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'SF weather'}, 'id': 'call_psv901bm', 'type': 'tool_call'}], usage_metadata={'input_tokens': 257, 'output_tokens': 18, 'total_tokens': 275})]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_psv901bm', 'function': {'arguments': '{"query

In [22]:
states = []
for state in abot.graph.get_state_history(thread):
    print(state)
    print("--")
    states.append(state)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', id='612d7ee1-8c31-4fd1-bb21-74e86f2f5b8c'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_psv901bm', 'function': {'arguments': '{"query":"SF weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 257, 'total_tokens': 275, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2ba8bf57-6764-4df5-931b-bcf67ea38043-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'SF weather'}, 'id': 'call_psv901bm', 'type': 'tool_call'}], usage_metadata={'input_tokens': 257, 'output_tokens': 18, 'total_tokens': 275})]}, next=('action',), config={'configurable': {'thread_id': '3', 'checkpoint_ns': '', 'checkpoint_id': '1efed0

In [ ]:
#  if contiue call
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'SF weather'}, 'id': 'call_yyopnyvw', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.775, \'lon\': -122.4183, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1739781807, \'localtime\': \'2025-02-17 00:43\'}, \'current\': {\'last_updated_epoch\': 1739781000, \'last_updated\': \'2025-02-17 00:30\', \'temp_c\': 10.4, \'temp_f\': 50.7, \'is_day\': 0, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/116.png\', \'code\': 1003}, \'wind_mph\': 5.6, \'wind_kph\': 9.0, \'wind_degree\': 286, \'wind_dir\': \'WNW\', \'pressure_mb\': 1020.0, \'pressure_in\': 30.13, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 100, \'cloud\': 50, \'feelslike_c\': 9.3, \'feelsl

In [28]:
states[0].values["messages"][-1].tool_calls[0]["id"]

'call_psv901bm'

In [30]:
_id = states[0].values["messages"][-1].tool_calls[0]["id"]
# toolMessage format
# ToolMessage(content="", name="tavily_search_results_json", tool_call_id="call_yyopnyvw")
state_update = {
    "messages": [
        ToolMessage(
            tool_call_id=_id,
            name="tavily_search_results_json",
            content="54 degree celcius",
        )
    ]
}
state_update

{'messages': [ToolMessage(content='54 degree celcius', name='tavily_search_results_json', tool_call_id='call_psv901bm')]}

In [35]:
branch_and_add = abot.graph.update_state(
    states[0].config, state_update, as_node="action"
)

In [42]:
states = []
for state in abot.graph.get_state_history(thread):
    print(state.values)
    print("--")
    states.append(state)

{'messages': [HumanMessage(content='Whats the weather in SF?', id='612d7ee1-8c31-4fd1-bb21-74e86f2f5b8c'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_psv901bm', 'function': {'arguments': '{"query":"SF weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 257, 'total_tokens': 275, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2ba8bf57-6764-4df5-931b-bcf67ea38043-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'SF weather'}, 'id': 'call_psv901bm', 'type': 'tool_call'}], usage_metadata={'input_tokens': 257, 'output_tokens': 18, 'total_tokens': 275}), ToolMessage(content='54 degree celcius', name='tavily_search_results_json', id='9570fcba-5970-438f-a71a-d062f62233cd', tool_call_

In [46]:
for event in abot.graph.stream(None, branch_and_add):
    for k, v in event.items():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in SF?', id='612d7ee1-8c31-4fd1-bb21-74e86f2f5b8c'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_psv901bm', 'function': {'arguments': '{"query":"SF weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 257, 'total_tokens': 275, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2ba8bf57-6764-4df5-931b-bcf67ea38043-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'SF weather'}, 'id': 'call_psv901bm', 'type': 'tool_call'}], usage_metadata={'input_tokens': 257, 'output_tokens': 18, 'total_tokens': 275}), ToolMessage(content='54 degree celcius', name='tavily_search_results_json', id='9570fcba-5970-438f-a71a-d062f62233cd', tool_call_

In [51]:
print(v["messages"][0].content)

The current temperature in San Francisco is 54 degrees Celsius, but I couldn't find any information on the current weather conditions. If you'd like to know more about the forecast for SF, I can try looking that up for you.

Would you like me to check the weather forecast for San Francisco?
